In [ ]:
### SVM and SVR

In [1]:
# IMPORTS
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
# from sklearn.cross_validation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
# INIT
number_samples = [5] + list(range(10, 101, 10)) + [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 4001, 1000))
number_samples = [128, 256, 512, 1024, 2048, 4096]
# number_samples = [50, 100, 200, 300, 400]
# number_samples = [400]
C_vec = list(np.arange(0.1, 10, 0.5))
max_pus_number, max_sus_number, num_sensors = 20, 4, 1600
IS_SENSORS = True
DUMMY_LOC_VALUE, DUMMY_POWER_VALUE = -110.0, -110.0

In [3]:
# LOAD DATA
num_columns = (num_sensors if IS_SENSORS else max_pus_number * 3 + 1) + max_sus_number * 3 + 2
cols = [i for i in range(num_columns)]
dataframe = pd.read_csv('../../java_workspace/research/spectrum_allocation/resources/data/' +
                        'dynamic_pus_1600sensor_30000_min10_max20PUs_min1_max4SUs_square100grid_splat_2021_10_28_15_33.txt', 
                        delimiter=',', header=None, names=cols)
dataframe_max = pd.read_csv('../../java_workspace/research/spectrum_allocation/resources/data/' +
                            'dynamic_pus_max_power_30000_min10_max20PUs_min1_max4SUs_square100grid_splat_2021_10_28_15_33.txt', delimiter=',', header=None)

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)
dataframe_max[dataframe_max.shape[1] - 1] = dataframe_max[dataframe_max.shape[1] - 1].astype(float)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1]]], axis=1,
                        ignore_index=True)
idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)

data_reg = np.concatenate((dataframe_tot.values[:, 0:dataframe_tot.shape[1]-3], 
                           dataframe_tot.values[:, dataframe_tot.shape[1]-1:dataframe_tot.shape[1]]), axis=1)
# data_reg[data_reg < -110.0] = -110.0
data_class = dataframe_tot.values[:, 0:dataframe_tot.shape[1]-1]
y_class_power = dataframe_tot.values[:, -1]
del dataframe, dataframe_max

In [3]:
# LOAD DATA for testbed
num_columns = (num_sensors if IS_SENSORS else max_pus_number * 3 + 1) + max_sus_number * 3 + 1
cols = [i for i in range(num_columns)]
dataframe = pd.read_csv('ML/data/testbed/' + 'su_ss_calibrate_shuffled', 
                        delimiter=',', header=None, names=cols)
dataframe.reset_index(drop=True, inplace=True)
data_reg = dataframe.values
data_reg[data_reg<-90] = -90

In [4]:
selected_columns = [4, 2, 16, 14]
droped_columns = []
for i in range(num_sensors):
    if i not in selected_columns:
        droped_columns.append(i)
data_reg = np.delete(data_reg, droped_columns, 1)
num_sensors = len(selected_columns)
num_columns = (num_sensors if IS_SENSORS else max_pus_number * 3 + 1) + max_sus_number * 3 + 2

In [7]:
data_reg.shape

(19126, 1604)

In [4]:
# Splitting data
# def split_data(data, train_samples):
#     num_inputs = data.shape[1] - 1
#     val_samples = round(train_samples/3)
#     X_train, y_train = data[0:train_samples, 0: num_inputs], data[0:train_samples, -1]
#     X_val, y_val = data[train_samples:train_samples+val_samples, 0: num_inputs],data[train_samples:train_samples+val_samples, -1]
#     X_test, y_test = data[train_samples:, 0: num_inputs], data[train_samples:, -1]
#     return X_train,X_val, X_test, y_train, y_val, y_test

def split_data(data: np.ndarray, train_samples):
    num_inputs = (max_sus_number - 1) * 3 + 2 + (max_pus_number * 3 
                                                 if not IS_SENSORS else num_sensors)
    val_samples = round(train_samples/5)
    test_samples = data.shape[0] - val_samples - train_samples
    #init arrays
    X_train = np.ones((train_samples, num_inputs), dtype=float) * DUMMY_LOC_VALUE
    X_val = np.ones((val_samples, num_inputs), dtype=float) * DUMMY_LOC_VALUE
    X_test = np.ones((test_samples, num_inputs), dtype=float) * DUMMY_LOC_VALUE
    # read values
    if not IS_SENSORS:
        # fill train
        for train_sample in range(train_samples):
            num_pus = int(data[train_sample, 0])
            num_sus = int(data[train_sample, 1 + num_pus * 3])
            X_train[train_sample, :num_pus * 3] = data[train_sample, 1:1 + num_pus * 3]#pus
            #sus except power of last su
            X_train[train_sample, max_pus_number * 3: (max_pus_number + num_sus) * 3 
                    - 1] = data[train_sample, 2 + num_pus * 3: 
                                1 + (num_pus + num_sus) * 3]
        # fill validation
        for val_sample in range(train_samples, train_samples + val_samples):
            num_pus = int(data[val_sample, 0])
            num_sus = int(data[val_sample, 1 + num_pus * 3])
            X_val[val_sample - train_samples, :num_pus * 3] = data[val_sample, 1:1 + num_pus * 3]
            X_val[val_sample - train_samples, max_pus_number * 3: 
                  (max_pus_number + num_sus) * 3 - 1] = data[val_sample, 2 + num_pus * 3:
                                                             1 + (num_pus + num_sus) * 3]
        # fill test
        for test_sample in range(train_samples + val_samples, 
                                 train_samples + val_samples + test_samples):
            num_pus = int(data[test_sample, 0])
            num_sus = int(data[test_sample, 1 + num_pus * 3])
            X_test[test_sample - (train_samples + val_samples), :num_pus * 3] = data[
                test_sample, 1:1 + num_pus * 3]
            X_test[test_sample - (train_samples + val_samples), max_pus_number * 3:
                   (max_pus_number + num_sus) * 3 - 1] = data[test_sample, 2 + num_pus * 3:
                                                              1 + (num_pus + num_sus) * 3]
    else:
#         X_train[:,:] = data[:train_samples, :data.shape[1]-1]
#         X_val[:,:] = data[train_samples: train_samples + val_samples, :data.shape[1]-1]
#         X_test[:,:] = data[train_samples + val_samples:, :data.shape[1]-1]
        # read sensors
        X_train[:, :num_sensors] = data[:train_samples, :num_sensors]
        X_val[:, :num_sensors] = data[train_samples: train_samples + val_samples,
                                         :num_sensors]
        X_test[:, :num_sensors] = data[train_samples + val_samples : , :num_sensors]
        #read sus
        for train_sample in range(train_samples):
            num_sus = int(data[train_sample, num_sensors])
            X_train[train_sample, num_sensors: num_sensors + num_sus * 3 - 1] = data[
                train_sample, num_sensors + 1:num_sensors + num_sus * 3]
            
        for val_sample in range(train_samples, train_samples + val_samples):
            num_sus = int(data[val_sample, num_sensors])
            X_val[val_sample - train_samples, num_sensors: num_sensors + num_sus * 3 - 1] =\
                data[val_sample, num_sensors + 1:num_sensors + num_sus * 3]
            
        for test_sample in range(train_samples + val_samples, 
                                 train_samples + val_samples + test_samples):
            num_sus = int(data[test_sample, num_sensors])
            X_test[test_sample - (train_samples + val_samples), num_sensors:
                   num_sensors + num_sus * 3 - 1] = data[test_sample, 
                                                         num_sensors + 1:num_sensors + num_sus * 3]

    
    y_train = data[0 : train_samples, -1]
    y_val = data[train_samples : train_samples + val_samples, -1]
    y_test = data[train_samples + val_samples:, -1]
    return np.asarray(X_train).astype(np.float32),  np.asarray(X_val).astype(np.float32),\
           np.asarray(X_test).astype(np.float32), np.asarray(y_train).astype(np.float32), \
           np.asarray(y_val).astype(np.float32), np.asarray(y_test).astype(np.float32)
#     return X_train,X_val, X_test, y_train, y_val, y_test

In [7]:
#utils
def false_analysis(y_test, y_pred):
    tp = sum(y_pred[y_test==1])
    fp = sum(y_pred) - tp
    return fp, sum(y_test) - tp

In [5]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_reg, 20)
print(data_reg[0])
print(X_train[0])
print(y_train[0])
print("***")
print(data_reg[20])
print(X_val[0])
print(y_val[0])
print("***")
print(data_reg[24])
print(X_test[0])
print(y_test[0])

[-77.697 -86.439 -77.45100000000001 ... nan nan 0.947]
[ -77.697  -86.439  -77.451 ... -110.    -110.    -110.   ]
0.947
***
[-88.33200000000001 -89.958 -79.385 ... nan nan -28.69]
[ -88.332  -89.958  -79.385 ... -110.    -110.    -110.   ]
-28.69
***
[-78.065 -75.17399999999999 -88.085 ... nan nan -18.64]
[ -78.065  -75.174  -88.085 ... -110.    -110.    -110.   ]
-18.64


In [ ]:
data_reg[]

In [ ]:
data_reg

In [13]:
## SVM(SVC)
average_class_diff_power = []
fp_mean_power = []
accuracy, f_score, false_positive, false_negative = [], [], [], []
best_c_lst = []
fp_penalty_coef, fn_penalty_coef = 1, 1
metric = "fp_min"  # {"accuracy", "fp_min"}
class_weight = {0:fp_penalty_coef/(fp_penalty_coef + fn_penalty_coef), 1:fn_penalty_coef/(fp_penalty_coef + fn_penalty_coef)}
best_c, bestsvcclassifier = None, None
for number_sample in number_samples:
    best_accuracy = -float('inf')
    best_fp = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_class, number_sample)
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svclassifier = SVC(kernel='rbf', C=c, class_weight = class_weight)
        svclassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svclassifier.predict(X_val)
        if metric == "accuracy":
            accuracy_tmp = metrics.accuracy_score(y_val, y_pred_val)
            if accuracy_tmp > best_accuracy:
                best_accuracy = accuracy_tmp
                best_c = c
                bestsvcclassifier = svclassifier
        elif metric == "fp_min":
            conf_mat = metrics.confusion_matrix(y_val, y_pred_val)
            fp_tmp = conf_mat[0][1] if len(conf_mat) == 2 else 0
            if fp_tmp < best_fp:
                best_fp = fp_tmp
                best_c = c
                bestsvcclassifier = svclassifier
                    
            
    best_c_lst.append(best_c)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvcclassifier.predict(X_test)
    
    #evaluating
    accuracy.append(round(metrics.accuracy_score(y_test, y_pred), 3))
    f_score.append(round(metrics.f1_score(y_true=y_test, y_pred=y_pred), 3))
    fp, fn = false_analysis(y_test, y_pred)
    false_positive.append(int(fp))
    false_negative.append(int(fn))
    
    #Power max calculations
    y_class_power_test = y_class_power[len(y_class_power)-X_test.shape[0]:]
    y_class_power_pred = np.zeros(y_class_power_test.size)
    max_power = max(y_class_power_test) + 10  # 10 is added to increase higher bound
    min_power = min(y_class_power_test) - 10  # 10 is subtracted to decrease lower bound
    for i in range(len(y_class_power_test)):
        h = max_power
        l = min_power
        while h - l > 0.5:
            mid = l + (h - l)/2;
            mid_norm = (mid - scaler_x.mean_[-1])/scaler_x.scale_[-1]
            X_test[i][-1] = mid_norm
            res_tmp = bestsvcclassifier.predict(X_test[i:i+1])
            if res_tmp[0]:
                l = mid
            else:
                h = mid
        y_class_power_pred[i] = l + (h - l)/2
    average_class_diff_power.append(round(np.mean(np.absolute(y_class_power_pred - y_class_power_test)), 3))
    fp_samples = np.zeros(len(y_class_power_pred), dtype=float)
    fp_samples[y_class_power_pred > y_class_power_test] = (y_class_power_pred - y_class_power_test)[y_class_power_pred > 
                                                                                                    y_class_power_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples:', number_sample, ', accuracy:', accuracy[-1], ', f_score:', f_score[-1], 
          ', fp:', fp,', fn:', fn, ', error:', average_class_diff_power[-1], 'fp_error:', fp_mean_power[-1])
del svclassifier

NameError: name 'data_class' is not defined

In [ ]:
print(average_class_diff_power)
print(fp_mean_power)

In [ ]:
### MAX_POWER ANAlysis
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, accuracy)
plt.xlabel('# training samples')
plt.ylabel('%')
plt.title('SVM: Classification Accuracy(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmAcc.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, f_score)
plt.xlabel('# training samples')
plt.ylabel('%')
plt.title('SVM: Classification F_score(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfscore.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, false_positive)
plt.xlabel('# training samples')
plt.ylabel('#')
plt.title('SVM: Classification FP(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfp.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, false_negative)
plt.xlabel('# training samples')
plt.ylabel('#')
plt.title('SVM: Classification FN(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfn.png')

In [19]:
number_samples

[50, 100, 200, 300, 400]

In [6]:
# SVR
average_reg_diff_power, best_c_reg_lst, fp_mean_power = [], [], []
best_c_reg, bestsvrclassifier =  None, None
# TODO: Having different penalties for fp and fn
for number_sample in number_samples:
    min_err = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_reg, number_sample)
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svrlassifier = SVR(kernel='rbf', C=c, degree=X_train.shape[1]+1)
        svrlassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svrlassifier.predict(X_val)
        err_tmp = np.mean(np.absolute(y_pred_val - y_val))
        if err_tmp < min_err:
            min_err = err_tmp
            best_c_reg = c
            bestsvrclassifier = svrlassifier
            
    best_c_reg_lst.append(best_c_reg)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvrclassifier.predict(X_test)
    
    
    #evaluating
    average_reg_diff_power.append(round(np.mean(np.absolute(y_test - y_pred)), 3))
    fp_samples = np.zeros(len(y_test), dtype=float)
    fp_samples[y_pred > y_test] = (y_pred - y_test)[y_pred > y_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples: ', number_sample, ' error: ', average_reg_diff_power[-1], 
          ', fp_error:', fp_mean_power[-1])

Number_samples:  128  error:  13.29 , fp_error: 6.356
Number_samples:  256  error:  12.697 , fp_error: 6.65
Number_samples:  512  error:  12.196 , fp_error: 6.435
Number_samples:  1024  error:  11.226 , fp_error: 6.096
Number_samples:  2048  error:  10.22 , fp_error: 5.651
Number_samples:  4096  error:  9.419 , fp_error: 5.312


In [11]:
# Linear SVR
average_reg_diff_power, best_c_reg_lst, fp_mean_power = [], [], []
best_c_reg, bestsvrclassifier =  None, None
for number_sample in number_samples:
    min_err = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_reg, number_sample)
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svrlassifier = LinearSVR(C=c, loss='epsilon_insensitive')
        svrlassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svrlassifier.predict(X_val)
        err_tmp = np.mean(np.absolute(y_pred_val - y_val))
        if err_tmp < min_err:
            min_err = err_tmp
            best_c_reg = c
            bestsvrclassifier = svrlassifier
            
    best_c_reg_lst.append(best_c_reg)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvrclassifier.predict(X_test)
    
    
    #evaluating
    average_reg_diff_power.append(round(np.mean(np.absolute(y_test - y_pred)), 3))
    fp_samples = np.zeros(len(y_test), dtype=float)
    fp_samples[y_pred > y_test] = (y_pred - y_test)[y_pred > y_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples: ', number_sample, ' error: ', average_reg_diff_power[-1], ', fp_error:', fp_mean_power[-1])

Number_samples:  128  error:  7.92 , fp_error: 3.702
Number_samples:  256  error:  8.361 , fp_error: 3.901
Number_samples:  512  error:  8.854 , fp_error: 4.193


/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base

Number_samples:  1024  error:  9.11 , fp_error: 4.485


/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base

Number_samples:  2048  error:  9.045 , fp_error: 4.693


/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base

Number_samples:  4096  error:  8.479 , fp_error: 4.389


/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [11]:
print(average_reg_diff_power)
print(fp_mean_power)

[9.689, 7.33, 6.8, 6.764, 6.65]
[1.127, 3.382, 2.999, 3.299, 3.788]


In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, average_class_diff_power)
plt.plot(number_samples, average_reg_diff_power, 'r--')
plt.xlabel('# training samples')
plt.ylabel('Diff(dB)')
plt.title('Average absolute difference power(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)

ax.set_yticks(np.arange(0,20, 2))
ax.set_ylim([2,20])
ax.set_xticks(np.arange(5,4100, 500))
# plt.grid(which='minor')
# plt.text(40, 50, '# Validation = 34k')
# plt.text(400, 45, '# Test = 34k')
plt.legend(['SVM', 'SVR'])
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_averag_powerSVMSVR.png')

In [ ]:
average_reg_diff_power_tot = []


In [ ]:
print(number_samples)

In [ ]:
average_reg_diff_power_tot.append(average_reg_diff_power)

In [ ]:
number_samples = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 200, 250, 300, 400, 500, 700, 1000, 2000, 3000, 4000]
average_reg_diff_power_tot = [[4.91, 4.915, 5.02, 7.274, 5.21, 5.476, 5.547, 5.448, 5.358, 5.486, 5.424, 5.585, 5.423, 5.369,
                               5.275, 5.22, 5.065, 5.058, 4.915, 4.873, 4.8, 4.787, 4.782], 
                              [4.739, 4.728, 4.858, 4.9, 4.761, 4.835, 4.898, 4.959, 4.806, 4.913, 4.945, 4.88, 4.8, 4.812,
                               4.756, 4.74, 4.764, 4.832, 4.729, 4.745, 4.728, 4.728, 4.731], 
                              [5.59, 5.33, 4.764, 4.731, 4.728, 4.923, 5.086, 4.798, 4.816, 5.035, 4.827, 4.842, 4.757, 4.796,
                               4.734, 4.748, 4.74, 4.734, 4.728, 4.729, 4.726, 4.726, 4.728], 
                              [4.791, 4.781, 4.732, 5.677, 4.757, 4.837, 4.841, 5.242, 4.81, 4.955, 4.972, 5.106, 4.801, 4.754,
                               4.764, 4.748, 4.757, 4.763, 4.737, 4.735, 4.737, 4.74, 4.744]]

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, average_reg_diff_power_tot[0])
plt.plot(number_samples, average_reg_diff_power_tot[1], 'r--')
plt.plot(number_samples, average_reg_diff_power_tot[2], 'g.-')
plt.plot(number_samples, average_reg_diff_power_tot[3], 'y->')
plt.xlabel('# training samples')
plt.ylabel('Diff(dB)')
plt.title('Average absolute difference power(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)

ax.set_yticks(np.arange(0,8, 2))
ax.set_ylim([2,8])
ax.set_xticks(np.arange(5,4100, 500))
# plt.grid(which='minor')
# plt.text(40, 50, '# Validation = 34k')
# plt.text(400, 45, '# Test = 34k')
plt.legend(['linear', 'rbf', 'poly', 'sigmoid'])
# plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_averag_powerSVMSVR.png')